## * Recommendation project for people who want to open a restaurant in Berlin,Germany.

#### 1) Introduction/Business Problem 

* **Introduction**
= Berlin is the one of the very trendy cities. Berlin has been rapidly changing a lot because many people are coming for leaving and working. Hence various resaurants are openining and closing. my purpose of the data analysis is to help people's business. 


* **Problem**
= Many people can possibly think that once they open a restaurant with very fancy and tasty food in Berlin, wherever it opens, it would be a success. but the problem is that in Berlin, there are already too many competitors on even one kind of food. Personally, I already saw some people struggling to find a proper solution or information to start with their business. but people tend to rely on the some information in internet, which is not reliable. 


* **Objective**
= Hence I would like to help people's business to avoid the worst scenario. Because once people starts with a own business, probably they might need to spend the cost to setup the restaurant.Therefore , I'm going to design the recommendation data analysis by using the Foursqure data and clustering technique,ML,  And Based on the data, I'm going to recommend the best spot for their business. Kreuzberg is currently quite trendy and popular town in Berlin for people to enjoy various cultures like foods. Therefore I'm going to focus on this town to start with this project. 


* **target audience**
= Even though this project is desigined for only people who want to open a restaurant in Kreuzberg of Berlin, Actually this approach way can be applied to anywhere.  

#### 2) Data

There are two big categories for the data, which are location data and venue data from Foursquare. these data will be utilized for the data analaysis when to recommend the places for users who who want to find out the nice places for their businesses.  

1. Berlin post codes, geo-location information (latitude, longitude)
: basically, the data includes geo location information of all berlin areas as well as post code information.
https://data.technologiestiftung-berlin.de/dataset/plz/en


2. Berlin traffic information 
: this data-set provides traffic information per street.
https://data.technologiestiftung-berlin.de/dataset/verkehrsmengen/en


2. Berlin map based on postcode information (to narrow down the range )
: actually the data is not utilized on the real code, however it is referred to narrow down the data range(area)
https://www.interkart.de/en/wall-maps/europe/germany/city-maps/berlin-postcode-area-map.html  


3. Berlin Kreuzberg map information to check
http://zip-code.en.mapawi.com/germany/10/kreisfreie-stadt-berlin/2/141/berlin-kreuzberg/10961/1330/


4. Foursquare venue information ( via API )
https://developer.foursquare.com/docs/api/venues/details

* test ( checking out the data )
= this data comes from the first data set, which is berlin post code information. In order to narrow down the data-set scale,I'll filter out other town information. 

In [1]:
import json
import pandas as pd
lst_dict = []


with open('./data/postcodes_berlin.geojson') as f:
    data = json.load(f)

df = pd.DataFrame(columns=['postcode','latitude','longitude'])

for feature in data['features']:
    # only check Kreuzberg area in Berlin 
    if feature['properties']['spatial_name'] in ('10961', '10963', '10965', '10967', '10969', '10997', '10999'):  
        for position in feature['geometry']['coordinates']:
            for i in position:
                df = df.append({'postcode': feature['properties']['spatial_name'],'latitude':i[1],'longitude':i[0]}, ignore_index=True)

df.shape

(1361, 3)

* Extracting only Kreuzberg of Berlin's post code information and getting unique values by groupping the post code data

In [2]:
df_group=df.groupby('postcode').mean().reset_index()
df_group

,postcode,latitude,longitude
0,10961,52.491796,13.394398
1,10963,52.499171,13.384151
2,10965,52.487955,13.389993
3,10967,52.491046,13.419179
4,10969,52.502517,13.401676
5,10997,52.500781,13.435210
6,10999,52.499876,13.421885


* Foursquare venue information based on the area,"Berlin, Kreuzberg"

In [60]:
CLIENT_ID = 'BDLXZJQYJRJT03ITMAD0QYWJC1SDRS124RRYJ1M1O2JEFOR4' # your Foursquare ID
CLIENT_SECRET = 'VLAQW1AH2IJE4WVSP34XUGNY00OMJPWDBNGSCNF4FQCKNO3T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
import requests

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            30)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
berlin_venues = getNearbyVenues(names=df_group['postcode'],
                                   latitudes=df_group['latitude'],
                                   longitudes=df_group['longitude']
                                  )



10961
10963
10965
10967
10969
10997
10999


In [63]:
berlin_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,10961,52.491796,13.394398,Nonne & Zwerg,52.492641,13.396822,Mediterranean Restaurant
1,10961,52.491796,13.394398,Space Hall,52.490746,13.394768,Record Shop
2,10961,52.491796,13.394398,Ocakbasi Holzkohlengrill Restaurant,52.493043,13.394863,Mediterranean Restaurant
3,10961,52.491796,13.394398,Cuccuma,52.490682,13.394192,Coffee Shop
4,10961,52.491796,13.394398,Chapter One,52.489762,13.395976,Coffee Shop
...,...,...,...,...,...,...,...
199,10999,52.499876,13.421885,Lasan,52.499842,13.418158,Kurdish Restaurant
200,10999,52.499876,13.421885,Biererei Bar & Vintage Cellar,52.500666,13.421093,Beer Bar
201,10999,52.499876,13.421885,Kisch&Co,52.500935,13.420686,Bookstore
202,10999,52.499876,13.421885,Habibi,52.501375,13.418990,Middle Eastern Restaurant


* **average daily traffic data**

1) str_name = street name

2) dtv = average daily traffic count 

In [7]:
import json
import pandas as pd
lst_dict = []


with open('./data/traffic_count_2014.geojson') as f:
    data_traffic = json.load(f)

df_traffic = pd.DataFrame(columns=['str_name','dtv','latitude','longitude'])
    
for feature in data_traffic['features']:
    # only check Kreuzberg area in Berlin 
    #if feature['properties']['spatial_name'] in ('10961', '10963', '10965', '10967', '10969', '10997', '10999'):  
    #print(feature['properties']['STR_NAME'])
    #print(feature['properties']['DTV'])

    for position in feature['geometry']['coordinates']:

            df_traffic = df_traffic.append(
                {'str_name': feature['properties']['STR_NAME'],
                 'dtv':feature['properties']['DTV'] ,
                 'latitude':position[1],
                 'longitude':position[0]}, ignore_index=True)

In [8]:
df_traffic['dtv'] = df_traffic.dtv.astype(float)
df_traffic.dtypes

str_name      object
dtv          float64
latitude     float64
longitude    float64
dtype: object

In [10]:
df_traffic_group=df_traffic.groupby(['str_name']).mean().reset_index()
df_traffic_group

,str_name,dtv,latitude,longitude
0,-,16160.178571,52.540296,13.636099
1,100 AD Charlottenburg Ausfahrt n Tegeler Weg,45798.000000,52.532825,13.289235
2,100 AD Funkturm Verb-Fb v 100 Norden n A 115,28375.000000,52.500659,13.274955
3,100 AK Schöneberg Ausf n Grazer D v 100 Innsbr,8820.000000,52.471152,13.347543
4,100 AK Schöneberg Begleitfahrbahn A 100 n Osten,11245.235294,52.474191,13.351451
...,...,...,...,...
1411,Zwickauer Damm,7020.000000,52.420945,13.481464
1412,ehemals 100 AS Schmargendorf Ausf n Dillenburg...,9496.000000,52.469575,13.306668
1413,ehemals 100 AS Schmargendorf Ausfahrt n Schild...,15470.000000,52.465825,13.312450
1414,ehemals 100 AS Schmargendorf Einf v Dillenburg...,8841.000000,52.469525,13.307206


* filtering the most crowded streets to narrow down the data range.

In [58]:
## test -- sampling 1 street 
#df_traffic_group[df_traffic_group['str_name'].str.contains('Halles')]  Hallesches Ufer
#df_traffic_group[df_traffic_group['str_name'].str.contains('Kott')] Kottbusser Damm
#df_traffic_group[df_traffic_group['str_name'].str.contains('Hasen')] Hasenheide
#df_traffic_group[df_traffic_group['str_name'].str.contains('Colum')] Columbiadamm
#df_traffic_group[df_traffic_group['str_name'].str.contains('Boelck')] Boelckestr.
#df_traffic_group[df_traffic_group['str_name'].str.contains('Leip')] Leipziger Str.
#df_traffic_group[df_traffic_group['str_name'].str.contains('Zoss')] Zossener Str.

df_traffic_group_filter = df_traffic_group[df_traffic_group['str_name'].str.contains('Zoss') | 
                 df_traffic_group['str_name'].str.contains('Leipziger Str') | 
                 df_traffic_group['str_name'].str.contains('Boelckestr') |
                 df_traffic_group['str_name'].str.contains('Columbiadamm') |
                 df_traffic_group['str_name'].str.contains('Hasenheide') |
                 df_traffic_group['str_name'].str.contains('Kottbusser Damm') |
                 df_traffic_group['str_name'].str.contains('Hallesches Ufer') | 
                 df_traffic_group['str_name'].str.contains('Oranienstr') | 
                 df_traffic_group['str_name'].str.contains('Prinzenstr') |
                 df_traffic_group['str_name'].str.contains('Gneisen') |
                 df_traffic_group['str_name'].str.contains('Oberbaumstr')
                ]
df_traffic_group_filter

,str_name,dtv,latitude,longitude
296,Boelckestr.,18420.576923,52.478477,13.377231
356,Columbiadamm,17512.857143,52.482459,13.405295
520,Gneisenaustr.,16627.500000,52.490912,13.397713
571,Hallesches Ufer,20346.206897,52.499718,13.382098
582,Hasenheide,22050.000000,52.487641,13.417636
738,Kottbusser Damm,20340.000000,52.492349,13.422467
778,Leipziger Str.,43288.392857,52.510509,13.391469
939,Oberbaumstr.,9630.000000,52.501030,13.443069
956,Oranienstr.,13032.857143,52.503225,13.412267
1027,Prinzenstr.,13410.000000,52.500317,13.408118


* representing data on the map

In [59]:
#BERLIN Kreuzberg position
latitude=52.4992
longitude=13.4307

import folium

map_berlin_kreuzberg = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng,postcode in zip(df_group['latitude'], df_group['longitude'], df_group['postcode']):
    label='{}'.format(postcode)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=100,
        popup=label,
        color='blue',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_berlin_kreuzberg)

for lat,lng,dtv in zip(df_traffic_group_filter['latitude'], df_traffic_group_filter['longitude'], df_traffic_group_filter['dtv']):
    label='{}'.format(dtv)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_berlin_kreuzberg)
    
    
map_berlin_kreuzberg


#### 3) Methodology 

Based on the traffic data and post code information including the coordinates data, we'll analayze the each neighborhood by using Foursquare data. the neighborhood data is utilized to analyze the people's tendency on the each area about where to go. And also, I'm going to use K means clustering to cluster area that have similar characteristics. Geologically, We'll find out the pattern of the data which is combination of clusters and neighborhood data.

In [65]:
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")
berlin_onehot['Neighborhood'] = berlin_venues['Neighborhood']
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]
berlin_onehot
berlin_onehot.shape

(204, 99)

In [66]:
berlin_grouped = berlin_onehot.groupby('Neighborhood').mean().reset_index()
berlin_grouped

,Neighborhood,African Restaurant,Art Gallery,Art Museum,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,...,Tea Room,Theater,Tibetan Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,10961,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.033333,0.000000
1,10963,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.000000,...,0.000000,0.076923,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000
2,10965,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.066667,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333
3,10967,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,...,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.000000
4,10969,0.000000,0.071429,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.035714,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000
5,10997,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.033333,0.100000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,10999,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.066667


* check out each neighborhood along with the top 10 most common venues

In [106]:
num_top_venues = 10

for hood in berlin_grouped['Neighborhood']:
    print("----"+hood+":postcode ----")
    temp = berlin_grouped[berlin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----10961:postcode ----
                      venue  freq
0        Italian Restaurant  0.13
1               Coffee Shop  0.10
2  Mediterranean Restaurant  0.07
3              Cocktail Bar  0.07
4          Sushi Restaurant  0.03
5         Korean Restaurant  0.03
6                    Market  0.03
7          Greek Restaurant  0.03
8              Gourmet Shop  0.03
9                     Plaza  0.03


----10963:postcode ----
               venue  freq
0               Café  0.12
1             Hostel  0.08
2     Science Museum  0.08
3            Theater  0.08
4        Supermarket  0.04
5           Pharmacy  0.04
6  French Restaurant  0.04
7               Park  0.04
8    Organic Grocery  0.04
9          Gastropub  0.04


----10965:postcode ----
                         venue  freq
0           Italian Restaurant  0.10
1             Greek Restaurant  0.07
2                          Bar  0.07
3                 Gourmet Shop  0.03
4  Eastern European Restaurant  0.03
5           Falafel Restaurant 

* set up a function to sort the venues in descending order.

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

* create the new dataframe and check out top 10 venues for each neighborhood.

In [108]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = berlin_grouped['Neighborhood']

for ind in np.arange(berlin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10961,Italian Restaurant,Coffee Shop,Cocktail Bar,Mediterranean Restaurant,Bookstore,Greek Restaurant,Wine Shop,Food & Drink Shop,Korean Restaurant,Falafel Restaurant
1,10963,Café,Hostel,Theater,Science Museum,Historic Site,Spa,Gastropub,French Restaurant,Hotel,Concert Hall
2,10965,Italian Restaurant,Greek Restaurant,Bar,Yoga Studio,Eastern European Restaurant,Music Venue,Mediterranean Restaurant,Market,Dumpling Restaurant,Falafel Restaurant
3,10967,Café,Pizza Place,Bar,Coffee Shop,Pastry Shop,Pub,Ramen Restaurant,Breakfast Spot,Ice Cream Shop,Russian Restaurant
4,10969,Bistro,Art Gallery,Nightclub,Dance Studio,Café,Convenience Store,Park,Eastern European Restaurant,Food & Drink Shop,Coffee Shop


* Cluster Neighborhoods

By using the K mean clustering, I'm going to approach the problem to figure out the way how to separate the each area based on the similar characteristics. And also I'm present the result of it on the map with existing data together

In [109]:
from sklearn.cluster import KMeans

kclusters = 3
berlin_grouped_clustering = berlin_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 0, 2, 1], dtype=int32)

In [110]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
berlin_merged = df_group

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
berlin_merged = berlin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='postcode')
berlin_merged

,postcode,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10961,52.491796,13.394398,1,Italian Restaurant,Coffee Shop,Cocktail Bar,Mediterranean Restaurant,Bookstore,Greek Restaurant,Wine Shop,Food & Drink Shop,Korean Restaurant,Falafel Restaurant
1,10963,52.499171,13.384151,0,Café,Hostel,Theater,Science Museum,Historic Site,Spa,Gastropub,French Restaurant,Hotel,Concert Hall
2,10965,52.487955,13.389993,1,Italian Restaurant,Greek Restaurant,Bar,Yoga Studio,Eastern European Restaurant,Music Venue,Mediterranean Restaurant,Market,Dumpling Restaurant,Falafel Restaurant
3,10967,52.491046,13.419179,2,Café,Pizza Place,Bar,Coffee Shop,Pastry Shop,Pub,Ramen Restaurant,Breakfast Spot,Ice Cream Shop,Russian Restaurant
4,10969,52.502517,13.401676,0,Bistro,Art Gallery,Nightclub,Dance Studio,Café,Convenience Store,Park,Eastern European Restaurant,Food & Drink Shop,Coffee Shop
5,10997,52.500781,13.435210,2,German Restaurant,Café,Bar,Cocktail Bar,Italian Restaurant,Ice Cream Shop,Bookstore,Spanish Restaurant,Food Court,Fried Chicken Joint
6,10999,52.499876,13.421885,1,Yoga Studio,Middle Eastern Restaurant,Coffee Shop,Gay Bar,Music Store,Mediterranean Restaurant,Lounge,Kurdish Restaurant,Wine Shop,Gourmet Shop


* check out the cluster position on the map

In [103]:
# set color scheme for the clusters
import matplotlib.cm as cm
import matplotlib.colors as colors

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [105]:
#BERLIN Kreuzberg position
latitude=52.4992
longitude=13.4307

import folium

map_berlin_kreuzberg = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat,lng,postcode in zip(df_group['latitude'], df_group['longitude'], df_group['postcode']):
    label='{}'.format(postcode)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=100,
        popup=label,
        color='blue',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_berlin_kreuzberg)

for lat,lng,dtv in zip(df_traffic_group_filter['latitude'], df_traffic_group_filter['longitude'], df_traffic_group_filter['dtv']):
    label='{}'.format(dtv)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=2,
        popup=label,
        color='black',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_berlin_kreuzberg)
    
for lat, lon, poi, cluster in zip(berlin_merged['latitude'], berlin_merged['longitude'], berlin_merged['postcode'], berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_berlin_kreuzberg)
    
    
map_berlin_kreuzberg

#### 4) Result 

Through the K-means cluster technique, I can see that there 5 major areas on the Kreuzberg, berlin to suggest the nice places. As you can see the map,  you can figure out that the red cluster, which is located in west-north area, Mostly, there are many cafes and bistros. Also, In the red cluster zone, we can figure out that there are historic sites , hostels, art galleries and theaters compared to other area(cluster).  

On the first cluster((post code 10965 - 10999), Mostly there are many nice Italian Restaurants and yoga places. On the second clusters(post code 10967 - 10997), which are located in east of Kreuzberg, there are many cafes and german restaurants. 

Overall, there are many cafes and bars over the all areas. famous places mostly are located in west area. 

According to the traffic information(black color points on the map), there are huge traffic volume around U-bahn(subway)stations such as Mockernbrucke, Kottbusser Tor, Schonleinstrasse and Gneisenaustrasse. On the cluster 0, there are almost 20000 ~ 40000 daily traffic amount which are the highest among the clusters. because there are famous venues on the area

#### 5) Discussion

Generally, there are already so many cafes and bars over the area of Kreuzberg. If a person has to choose a category of restaurant, it would be great to avoid the cafe and bars. the details of each cluster are explained from the data perspective.

2) Cluster 0 (red color cluster)
There are many venus related to culture like museum, theater compared to other clusters. it means that if a new business has to be started, it would be great to have a business that people can quickly use. 

* daily traffic amount : around 20k ~ 40k 

----10963:postcode ----
               venue  freq
0               Café  0.12
1             Hostel  0.08
2     Science Museum  0.08
3            Theater  0.08
4        Supermarket  0.04
5           Pharmacy  0.04
6  French Restaurant  0.04
7               Park  0.04
8    Organic Grocery  0.04
9          Gastropub  0.04


1) Cluster 1 (purple color cluster)
On the middle of Kreuzberg area, cluster 1 has been set up by k-mean clusters. one of the most remarkable things on the cluster is that there are many kinds of restaurants such as korean food, japanese food, greece food as well as italian food. because this area is located in the center of Kreuzberg, people would easily recognize the resaurants. And also, people tend to use the sport club like Yoga studio a lot in this area according to the 10999 post code area. it would be great to have more sport clubs in this area. people might love that if there is a new nice sport club around the area.

* daily traffic amount : around 13k ~ 16k 

----10961:postcode ----
                      venue  freq
0        Italian Restaurant  0.13
1               Coffee Shop  0.10
2  Mediterranean Restaurant  0.07
3              Cocktail Bar  0.07
4          Sushi Restaurant  0.03
5         Korean Restaurant  0.03
6                    Market  0.03
7          Greek Restaurant  0.03
8              Gourmet Shop  0.03
9                     Plaza  0.03

----10999:postcode ----
                       venue  freq
0                Yoga Studio  0.07
1                Coffee Shop  0.07
2  Middle Eastern Restaurant  0.07
3       Fast Food Restaurant  0.03
4                    Gay Bar  0.03
5      General Entertainment  0.03
6               Gourmet Shop  0.03
7         Kurdish Restaurant  0.03
8                     Lounge  0.03
9   Mediterranean Restaurant  0.03

2) Cluster 2 (light green color cluster)
Becase of the reason that this area(cluster) a bit close to the S-Bahn Warschauer station, there are also many bars and cafes. as you can see the data below, people tend to visit German restaurants compared to other cluster areas.  

* daily traffic amount : around 10k ~ 20k 

----10967:postcode ----
                 venue  freq
0                 Café  0.10
1          Pizza Place  0.10
2                  Bar  0.07
3          Coffee Shop  0.07
4        Deli / Bodega  0.03
5  Indie Movie Theater  0.03
6       Ice Cream Shop  0.03
7      Organic Grocery  0.03
8          Pastry Shop  0.03
9                  Pub  0.03

----10997:postcode ----
                venue  freq
0   German Restaurant  0.10
1                 Bar  0.10
2                Café  0.10
3  Italian Restaurant  0.07
4        Cocktail Bar  0.07
5      Ice Cream Shop  0.07
6         Cheese Shop  0.03
7      Breakfast Spot  0.03
8          Food Court  0.03
9  Spanish Restaurant  0.03

#### 6) Conclusion

If people have to find a nice place to open a new restaurant in Kreuzberg, Berlin. people have to consider major 3 factors as below.

1) How close is it to U-bahn station?
2) How many restaurants are already opened on the area on a certain category that you want?
3) Why do people visit the area?

As I reviewd the data on the discussion, Over the area of Kreuzberg, there are already so many cafes and bars as well as restaurants. Once we look at the west-north area of map, there are some famous sites like historical builings. Hence it would be great to open a shop or restaurant which is very helpful for tourlists. they might want to use the shop on a certain purpose very fast to save the time. people here might have the tendency to move quickly and to be tourlists. 

When it comes to the center area (cluster 1), even though many people go through the area, there are not many sport clubs and culture-oriented buildings.In this area, it would be great to open a shop or restaurant for the potential consumers who want more healthy food and complex business that has a food shop and sport club. 

In conclusion,  In the cluster 0(red), it would be great to open a shop or restaurant by the very unique and necessary stuffs for the tourlists. In the cluster 1(purple), it would be great to open a shop or restaurant that can provide very healthy foods that people who like sports would love to.  In the cluster 2(light green), there are already many fancy restaurants which are german. Hence it would be great to open a shop or restaurant that provide the different international foods.